In [1]:
from collections import defaultdict
import numpy as np
from math import sqrt
from multiprocessing import Pool

In [5]:
# UNDO :  common calculate   raw_id2inner_id  pearson_similarity  predict_ratings

In [30]:
def common(x1,x2):
    # find common ratings
    common = (x1 != 0) & (x2 != 0)
    new_x1 = x1[common]
    new_x2 = x2[common]
    return new_x1, new_x2

In [31]:
x = np.array([1,2,3,0])
x1 = np.array([1,0, 4, 6])
c = (x != 0) & (x1 != 0)
x[c], x1[c]

(array([1, 3]), array([1, 4]))

In [2]:
train_set = open("ml-1m/ratings.dat").readlines()

In [31]:
unique_users = set([i.split("::")[0] for i in train_set])
sorted_users = [str(i) for i in np.sort(np.array(list(unique_users)).astype(int))]

In [32]:
user2id = {user: idx for idx, user in enumerate(sorted_users)}
id2user = {idx: user for user, idx in user2id.items()}

In [33]:
unique_items = set([i.split("::")[1] for i in train_set])
sorted_items = [str(i) for i in np.sort(np.array(list(unique_items)).astype(int))]

In [34]:
item2id = {item: idx for idx, item in enumerate(sorted_items)}
id2item = {idx: item for item, idx in item2id.items()}

In [35]:
data = defaultdict(dict)
for i in train_set:
    user = i.split("::")[0]
    item = i.split("::")[1]
    rating = i.split("::")[2]
    user_id = user2id[user]
    item_id = item2id[item]
    data[user_id].update(dict(zip([item_id], [int(rating)])))

In [8]:
data1 = defaultdict(dict)
user2id1 = {}
item2id1 = {}
index_user = 0
index_item = 0

for i in train_set:
    user = i.split("::")[0]
    item = i.split("::")[1]
    rating = i.split("::")[2]
    try:
        user_id = user2id1[user]
    except KeyError:
        user_id = index_user
        user2id1[user] = index_user
        index_user += 1
    try:
        item_id = item2id1[item]
    except KeyError:
        item_id = index_item
        item2id1[item] = index_item
        index_item += 1
        
    data1[user_id].update(dict(zip([item_id], [int(rating)])))

In [9]:
example = defaultdict(dict)
for key in range(200):
#    key = str(key)
    example[key] = data[key]

In [10]:
data = example

In [11]:
num = len(data.keys())
# sorted_keys = [str(i) for i in np.sort(np.array(list(data.keys())).astype(int))]

In [12]:
def cosine_similarity(dicts, x1, x2):  # only common ratings
    prod = denom1 = denom2 = 0
    for item in dicts[x1]:
        if item in dicts[x2]:
            prod += dicts[x1][item] * dicts[x2][item]
            denom1 += dicts[x1][item] ** 2
            denom2 += dicts[x2][item] ** 2
    try:
        return prod / sqrt(denom1 * denom2)
    except ZeroDivisionError:
        return 0

In [127]:
def cosine_similarity(dicts, x1, x2):
    prod = 0
    for item in dicts[x1]:
        if item in dicts[x2]:
            prod += dicts[x1][item] * dicts[x2][item]
    denom1 = np.sum(np.power(list(dicts[x1].values()), 2))
    denom2 = np.sum(np.power(list(dicts[x2].values()), 2))
    try:
        return prod / sqrt(denom1 * denom2)
    except ZeroDivisionError:
        return 0

In [21]:
cosine_similarity(data, 1, 100)

0.9766596404789609

In [103]:
n = 20

In [98]:
def map_func(i):
    user_ids = [user2id[user] for user in sorted_users[:n]]
    return [cosine_similarity(data, i, other) for other in user_ids]

In [85]:
def get_sim(n, parallel=False, n_jobs=4):
    if not parallel:
        user_ids = [user2id[user] for user in sorted_users[:n]]
        whole = np.array([[cosine_similarity(data, i, other) for other in user_ids] for i in user_ids])
    else:
        with Pool(processes=n_jobs) as p:
            user_ids = [user2id[user] for user in sorted_users[:n]]
            whole = p.map(map_func, user_ids)
            whole = np.array(whole)
    return whole

In [86]:
def get_sim2(n, parallel=False, n_jobs=4):
    if not parallel:
        sim = np.zeros((n, n))
        user_ids = [user2id[user] for user in sorted_users[:n]]
        for i in user_ids:
            sim[i, i] = 1.0
            for j in user_ids[i+1:]:
                sim[i, j] = cosine_similarity(data, i, j)
                sim[j, i] = sim[i, j]
    else:
        with Pool(processes=n_jobs) as p:
            user_ids = [user2id[user] for user in sorted_users[:n]]
            whole = p.map(map_func, user_ids)
            whole = np.array(whole)
    return sim

In [95]:
%timeit res = get_sim(n=200, parallel=False)

1 loop, best of 3: 1.13 s per loop


In [96]:
%timeit sim = get_sim2(n=200, parallel=False)

1 loop, best of 3: 545 ms per loop


In [83]:
res = get_sim(n=20, parallel=False)
res.shape

(20, 20)

In [88]:
sim = get_sim2(n=20, parallel=False)
sim.shape

(20, 20)

In [92]:
np.all(res == sim)

True

In [164]:
def recommend(user, n_neighbors, n_rec):
    reco = {}
    simSums = defaultdict(lambda : 0)
    user_id = rawid2innerid[user]
    neighbors_id = np.argsort(sim[user_id][::-1][1: n_neighbors + 1])
    for neighbor_id in neighbors_id:
        neighbor_raw = innerid2rawid[neighbor_id]
        ssim = sim[user_id, neighbor_id]
        for item in data[neighbor_raw]:
            if item not in reco:
                reco[item] = ssim * data[neighbor_raw][item]
            else:
                reco[item] += ssim * data[neighbor_raw][item]
            simSums[item] += ssim
#    rank = sorted(reco.items(), key=lambda x: x[1], reverse=True)[1: n_rec]
    rank = [(item, rating / simSums[item]) for item, rating in reco.items()]
    rank.sort(key=lambda x: x[1], reverse=True)
    return rank[:n_rec]

In [167]:
recommend("10", 10, 10)

[('2431', 5.0),
 ('248', 5.0),
 ('2236', 5.0),
 ('2640', 5.0),
 ('918', 5.0),
 ('1023', 5.0),
 ('29', 5.0),
 ('1831', 5.0),
 ('1016', 5.0),
 ('916', 5.0)]